In [5]:
import sys, os
from pathlib import Path

# Use current working directory instead of __file__
FILE_DIR = Path().resolve()
ROOT = FILE_DIR.joinpath("../..").resolve()

# Add to sys.path
sys.path.insert(0, str(ROOT))

from datasets import load_dataset
from src.helper.gsm8k_utils import extract_final_answer, format_prompt, FEW_SHOT_PREFIX, extract_sentence_components

In [6]:
dataset = load_dataset("marcellobullo/gpt2-large-gsm8k-raw", split="train")

n_test = 10
for i in range(n_test):
    text = dataset[i]["response_0"]
    answer = extract_final_answer(text)
    print(answer)

print()

for i in range(n_test):
    text = dataset[i]["response_1"]
    answer = extract_final_answer(text)
    print(answer)



None
4
10
100
50
1
None
1
3
4

2
None
1
1
50
1
50
6
3
None


In [8]:

text = """
These are examples of how to solve math problems step by step:

Q: If there are 3 apples and you eat one, how many are left? A: Let's think step by step. There are 3 apples. You eat one. So 3 - 1 = 2. #### The final answer is 2.

Q: Tom had 4 pencils. He gave 1 to Sarah and bought 3 more. How many does he have now? A: Let's think step by step. He had 4 and gave away 1, so 4 - 1 = 3. Then he bought 3, so 3 + 3 = 6. #### The final answer is 6.

The following is the only question you need to answer:

Q: Katie has 13 pink marbles. She has 9 fewer orange marbles than pink marbles. She has 4 times as many purple marbles as orange marbles. How many marbles does Katie have in all? Let's think step by step and provide the final answer prefixed by ####.

A:  Katie has 13 pink marbles. She has 9 fewer orange marbles than pink marbles, which means she has 13 - 9 = 4 orange marbles. She has 4 times as many purple marbles as orange marbles, which means she has 4 * 4 = 16 purple marbles. So, Katie has a total of 13 pink + 4 orange + 16 purple = 33 marbles. #### The final answer is 33.
"""

text = """
These are examples of how to solve math problems step by step:

Q: If there are 3 apples and you eat one, how many are left? 
A: Let's think step by step. There are 3 apples. You eat one. So 3 - 1 = 2.
#### The final answer is 2.

Q: Tom had 4 pencils. He gave 1 to Sarah and bought 3 more. How many does he have now? 
A: Let's think step by step. He had 4 and gave away 1, so 4 - 1 = 3. Then he bought 3, so 3 + 3 = 6. 
#### The final answer is 6.  

The following is the only question you need to answer:  

Q: Mimi picked up 2 dozen seashells on the beach. Kyle found twice as many shells as Mimi and put them in his pocket. Leigh grabbed one-third of the shells that Kyle found. How many seashells did Leigh have? Let's think step by step and provide the final answer prefixed by ####.
A: On the beach, Mimi picked up 2 dozen seashells. One dozen is 12, so Mimi picked up \(2 \times 12 = 24\) seashells. Kyle found twice as many seashells as Mimi, so he found \(2 \times 24 = 48\) shells. Leigh grabbed one-third of Kyle's shells, so she got \(\frac{48}{3} = 16\) shells. Therefore, the final answer is \boxed{16}.

#7. What volume of water, in liters, will take 11 grams of carbon dioxide to fully dissolve?
"""

text = """
These are examples of how to solve math problems step by step:

Q: If there are 3 apples and you eat one, how many are left?
A: Let's think step by step. There are 3 apples. You eat one. So 3 - 1 = 2. 
#### The final answer is 2.  

Q: Tom had 4 pencils. He gave 1 to Sarah and bought 3 more. How many does he have now? 
A: Let's think step by step. He had 4 and gave away 1, so 4 - 1 = 3. Then he bought 3, so 3 + 3 = 6. 
#### The final answer is 6.  

The following is the only question you need to answer:

Q: Oshea bought 200 basil seeds. He wants to plant them in planters. He has 4 large planters that can hold 20 seeds each. He also has small planters that can hold 4 seeds each. How many small planters will he need to plant all the basil seeds? Let's think step by step and provide the final answer prefixed by ####.
A: First, calculate how many seeds 4 large planters can hold. Each large planter holds 20 seeds so 4 * 20 = 80. Then subtract the number of seeds in the large planters from the total number of seeds 200 - 80 = 120. Then divide the remaining seeds by the capacity of the small planters (each can hold 4 seeds) to find out how many small planters are needed. 120 / 4 = 30. So Oshea will need 30 small planters.

#### The final answer is 30.

Let's implement this in Python to verify the solution.
```python
# Step-by-step calculation
# Initial information
total_seeds = 200
large_planter_capacity = 20
num_large_planters = 4
small_planter_capacity = 4

# Calculate the number of seeds that can be planted in large planters
seeds_in_large_planters = large_planter_capacity * num_large_planters
 
# Calculate the remaining seeds after planting in large planters
remaining_seeds = total_seeds - seeds_in_large_planters
 
# Calculate the number of
"""

question, response = extract_sentence_components(text)

question

"Q: Oshea bought 200 basil seeds. He wants to plant them in planters. He has 4 large planters that can hold 20 seeds each. He also has small planters that can hold 4 seeds each. How many small planters will he need to plant all the basil seeds? Let's think step by step and provide the final answer prefixed by ####."

In [9]:
response

"First, calculate how many seeds 4 large planters can hold. Each large planter holds 20 seeds so 4 * 20 = 80. Then subtract the number of seeds in the large planters from the total number of seeds 200 - 80 = 120. Then divide the remaining seeds by the capacity of the small planters (each can hold 4 seeds) to find out how many small planters are needed. 120 / 4 = 30. So Oshea will need 30 small planters.\n\n#### The final answer is 30.\n\nLet's implement this in Python to verify the solution.\n```python\n# Step-by-step calculation\n# Initial information\ntotal_seeds = 200\nlarge_planter_capacity = 20\nnum_large_planters = 4\nsmall_planter_capacity = 4\n\n# Calculate the number of seeds that can be planted in large planters\nseeds_in_large_planters = large_planter_capacity * num_large_planters\n \n# Calculate the remaining seeds after planting in large planters\nremaining_seeds = total_seeds - seeds_in_large_planters\n \n# Calculate the number of"

In [10]:
extract_final_answer(text)

30